In [1]:
from chromalab.observer import Observer, Cone
from chromalab.spectra import Spectra, Illuminant
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import PolyCollection
from tqdm import tqdm
%matplotlib widget

from skimage.draw import polygon

ModuleNotFoundError: No module named 'chromalab'

In [ ]:

def dichromat_color_solid(ms_responses, wavelengths, rgb2rgb):
    """
    input:
    - responses: has shape (2, n)
    - rgb2rgb: associated dichromat color mapping
    - wavelneghts - set of wavelengths
    return:
    - vertices - optimal color vertices in responses[0] and responses[1]
    vertex_colors - optimal colors as perceived by a dichromat
    """
    # generate a list of optimal color vertices from the locus

    vertices = np.zeros((n * 3, 2))
    vertex_colors = np.zeros((n * 3, 3))

    points = np.copy(ms_responses).T # generating vectors

    for i in range(1, n + 1):
        # fills up vertices from 1 to n
        vertices[i] = vertices[i - 1] + points[i - 1]

        # fill in the reflectance
        reflectance_data = np.zeros(n)
        for j in range(i):
            # from black to white point
            reflectance_data[j] = 1 # give 1 to all generating vectors associating with the particular position
        reflectance = Spectra(wavelengths=wavelengths, data=reflectance_data)
        vertex_colors[i] = reflectance.to_rgb(illuminant)
        vertex_colors[i] = rgb2rgb @ vertex_colors[i]
        s = ms_responses[:,0]

    for i in range(1, n):
        # fills up vertices from n+1 to 2n
        vertices[i + n] = vertices[i + n - 1] - points[i - 1]
        reflectance_data = np.zeros(n)
        for j in reversed(range(i, n)):

            # from black to white point
            reflectance_data[j] = 1 # give 1 to all generating vectors associating with the particular position
        reflectance = Spectra(wavelengths=wavelengths, data=reflectance_data)
        vertex_colors[i + n] = reflectance.to_rgb(illuminant)
        vertex_colors[i + n] = rgb2rgb @ vertex_colors[i + n]

    # minmax normalize vertex colors
    vertex_colors[:, 0] = (vertex_colors[:, 0] - np.min(vertex_colors[:, 0])) / (np.max(vertex_colors[:, 0]) - np.min(vertex_colors[:, 0]))
    vertex_colors[:, 1] = (vertex_colors[:, 1] - np.min(vertex_colors[:, 1]))/ (np.max(vertex_colors[:, 1])- np.min(vertex_colors[:, 1]))
    vertex_colors[:, 2] = (vertex_colors[:, 2]- np.min(vertex_colors[:, 2]) )/ (np.max(vertex_colors[:, 2])- np.min(vertex_colors[:, 2]))

    # normalize vertex positions to (1,1)
    vertices[:, 0] =  (vertices[:, 0] - np.min(vertices[:, 0])) / (np.max(vertices[:, 0]) - np.min(vertices[:, 0]))
    vertices[:, 1] =  (vertices[:, 1] - np.min(vertices[:, 1])) / (np.max(vertices[:, 1]) - np.min(vertices[:, 1]))
    return vertices, vertex_colors

## Protanopia

In [2]:
# Cone responses of a protanopic viewer
wavelengths = np.arange(390, 701, 1)
n = len(wavelengths)
standard_dichromat = Observer.dichromat(wavelengths)
illuminant = Illuminant.get("ISO 7589 Photographic Daylight").interpolate_values(wavelengths)
ms_responses = np.vstack((standard_dichromat.sensors[0].data,
                           standard_dichromat.sensors[1].data))

T_protanopia = np.array([
    [0.170556992, 0.829443014, 0],
    [0.170556991, 0.829443008, 0],
    [-0.004517144, 0.004517144, 1]
])

NameError: name 'np' is not defined

In [ ]:
vertices, vertex_colors = dichromat_color_solid(ms_responses, wavelengths, T_protanopia)

# Plot each vertex with its RGB color on a 2D grid
fig, ax = plt.subplots()
for i, vertex in enumerate(vertices):
    ax.scatter(vertex[0], vertex[1], color=vertex_colors[i])
    # Display RGB values near the vertex

# Customize the plot
ax.set_aspect('equal')
plt.xlabel("S")
plt.ylabel("M")
plt.title("Protanopia optimal colors")
plt.grid(True)
plt.show()

## Deuteranope

In [ ]:
# Cone responses of a deuteranopic viewer
wavelengths = np.arange(390, 701, 1)
n = len(wavelengths)

illuminant = Illuminant.get("ISO 7589 Photographic Daylight").interpolate_values(wavelengths)
deuteranope = Observer.deuteranope(wavelengths, illuminant)
ls_responses = np.vstack((deuteranope.sensors[0].data,
                           deuteranope.sensors[1].data))

T_deuteranopia = np.array([
    [0.33066007, 0.66933993, 0],
    [0.33066007, 0.66933993, 0],
    [-0.02785538, 0.02785538, 1]
])

In [ ]:
vertices, vertex_colors = dichromat_color_solid(ls_responses, wavelengths, T_deuteranopia)

# Plot each vertex with its RGB color on a 2D grid
fig, ax = plt.subplots()
for i, vertex in enumerate(vertices):
    ax.scatter(vertex[0], vertex[1], color=vertex_colors[i])
    # Display RGB values near the vertex

# Customize the plot
ax.set_aspect('equal')
plt.xlabel("S")
plt.ylabel("L")
plt.title("Deuteranopia optimal colors")
plt.grid(True)
plt.show()

## Tritanopia

In [ ]:
# Cone responses of a tritanopic viewer
wavelengths = np.arange(390, 701, 1)
n = len(wavelengths)

illuminant = Illuminant.get("ISO 7589 Photographic Daylight").interpolate_values(wavelengths)
tritanope = Observer.tritanope(wavelengths, illuminant)
lm_responses = np.vstack((tritanope.sensors[0].data,
                           tritanope.sensors[1].data))

T_tritanopia = np.array([
    [1, 0.1273989, -0.1273989],
    [0, 0.8739093, 0.1260907],
    [0, 0.8739093, 0.1260907]
])

In [ ]:
vertices, vertex_colors = dichromat_color_solid(lm_responses, wavelengths, T_tritanopia)

# Plot each vertex with its RGB color on a 2D grid
fig, ax = plt.subplots()
for i, vertex in enumerate(vertices):
    ax.scatter(vertex[0], vertex[1], color=vertex_colors[i])
    # Display RGB values near the vertex

# Customize the plot
ax.set_aspect('equal')
plt.xlabel("M")
plt.ylabel("L")
plt.title("Tritanopia optimal colors")
plt.grid(True)
plt.show()